### Install Libraries

In [ ]:
pip install pytube


In [ ]:
!pip install moviepy

In [ ]:
pip install git+https://github.com/openai/whisper.git 

In [ ]:
pip install openai

In [ ]:
pip install langchain-openai

In [ ]:
pip install langchain_community

In [ ]:
! pip install -U git+https://github.com/linto-ai/whisper-timestamped

In [ ]:
! pip install langchain_experimental

In [ ]:
! pip install jq

In [ ]:
! pip install sentence-transformers

In [ ]:
! pip install faiss-cpu

In [ ]:
! pip install opencv-python

### Import Libraries

In [213]:
from pytube import YouTube as pyt
import os
from moviepy.editor import *
from moviepy.editor import AudioFileClip
import moviepy.editor as mp
import whisper
import whisper_timestamped as whisper
from openai import OpenAI
from base64 import b64encode
import requests
from openai import AzureOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import JSONLoader
from langchain_experimental.text_splitter import SemanticChunker
from transformers import AutoTokenizer, AutoModel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
import json
import torch
import regex as re
import time
import cv2

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
path = f'{os.getcwd()}/video_toBlog_transcriptor'
url = 'https://www.youtube.com/watch?v=zduSFxRajkE'

### Download the Video

In [215]:
# link of the video to be downloaded 
link = "https://www.youtube.com/watch?v=zduSFxRajkE"

def Download(link):
    yt = pyt(link)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path=f"{path}/original_files/video", filename="video_file.mp4")
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_stream.download(output_path=f"{path}/original_files/audio", filename="audio_file.mp4")
    print("Download is completed successfully")
    
def get_title(url):
    yt = pyt(url)
    return yt.title

title = get_title("https://www.youtube.com/watch?v=zduSFxRajkE")

Download(link)

Download is completed successfully


### Chunk video into multiple files based on chunk size

In [221]:
def split_audio(file_path, chunk_size_mb=12, output_folder=f"{path}/split_chunks"):
    global split_audio_return
    
    if not os.path.exists(file_path):
        print(f"File {file_path} not found.")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"File size: {file_size_mb:.2f} MB")

    if file_size_mb <= chunk_size_mb:
        print("File size is within the limit. No need to split.")
        return
    else:
        split_audio_return = True

    clip = AudioFileClip(file_path)
    total_duration = clip.duration
    chunk_duration = (chunk_size_mb / file_size_mb) * total_duration

    # Split the audio
    start = 0
    part = 1
    while start < total_duration:
        end = min(start + chunk_duration, total_duration)
        chunk = clip.subclip(start, end)
        chunk_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(file_path))[0]}_part{part}.mp4")
        chunk.write_audiofile(chunk_filename, bitrate="64k", codec="aac")

        print(f"Created chunk: {chunk_filename}")

        start = end
        part += 1

    clip.close()

In [222]:
# os.path.exists("Lets build the GPT Tokenizer.mp4")
split_audio(f"{path}/original_files/audio/audio_file.mp4")

File size: 46.61 MB
MoviePy - Writing audio in /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part1.mp4


MoviePy - Done.
Created chunk: /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part1.mp4
MoviePy - Writing audio in /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part2.mp4


MoviePy - Done.
Created chunk: /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part2.mp4
MoviePy - Writing audio in /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part3.mp4


MoviePy - Done.
Created chunk: /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part3.mp4
MoviePy - Writing audio in /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part4.mp4


MoviePy - Done.
Created chunk: /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/split_chunks/audio_file_part4.mp4


### Transcribing with Whisper & Writing to JSON File(s)

In [226]:
def process_audio_file(output_folder=f"{path}/transcript_json"):
    input_folder_chunks = f'{path}/split_chunks'
    for filename in os.listdir(f'{path}/split_chunks'):
        print(filename)
        if filename.endswith(".mp4"):
            audio = whisper.load_audio(f'{path}/split_chunks/{filename}')
            model = whisper.load_model("base")

            result = whisper.transcribe(model, audio, language="en")
            json_filename = f"{os.path.splitext(filename)[0]}_transcript.json"
            output_path = os.path.join(output_folder, json_filename)
            # print(json.dumps(result, indent = 2, ensure_ascii = False))
            with open(output_path, 'w') as f:
                json.dump(result['segments'], f, indent=4)
    return result


In [227]:
res = process_audio_file()

audio_file_part1.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 206351/206351 [01:29<00:00, 2309.67frames/s]


audio_file_part4.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 182381/182381 [01:29<00:00, 2048.12frames/s]


.ipynb_checkpoints
audio_file_part3.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 206351/206351 [01:41<00:00, 2036.32frames/s]


audio_file_part2.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 206351/206351 [01:36<00:00, 2128.73frames/s]


### Cleaning & Concatenating the Transcription JSON files

In [228]:
def extract_part_number(filename):
    match = re.search(r'part(\d+)', filename)
    return int(match.group(1)) if match else 0

def process_file(filepath, max_id, last_end_time):
    with open(filepath, 'r') as file:
        data = json.load(file)
        time_adjustment = last_end_time - float(data[0]['start']) if last_end_time else 0
        new_data = []
        for entry in data:
            new_entry = {
                'id': max_id + 1,
                'start': float(entry['start']) + time_adjustment,
                'end': float(entry['end']) + time_adjustment,
                'text': entry['text']
            }
            new_data.append(new_entry)
            max_id += 1
        return new_data, max_id, new_data[-1]['end'] if new_data else last_end_time

def process_transcripts(directory_path, split_audio_return):
    files = os.listdir(directory_path)
    combined_data = []
    max_id = -1
    last_end_time = 0.0

    if split_audio_return:
        sorted_files = sorted(
            [file for file in files if file.startswith('audio') and file.endswith('.json')],
            key=extract_part_number
        )
    else:
        sorted_files = [file for file in files if file.endswith('.json') and not 'part' in file]

    for filename in sorted_files:
        full_path = os.path.join(directory_path, filename)
        processed_data, max_id, last_end_time = process_file(full_path, max_id, last_end_time)
        combined_data.extend(processed_data)

    # Output the combined data to a new JSON file
    output_path = os.path.join(directory_path, 'combined_data.json')
    with open(output_path, 'w') as outfile:
        json.dump(combined_data, outfile, indent=4)

In [229]:
process_transcripts(f"{path}/transcript_json", True)

In [230]:
def full_transcript(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    transcript = ""
    for entry in data:
        transcript += entry['text']
    return transcript

In [231]:
video_transcript = full_transcript(f"{path}/transcript_json/combined_data.json")

### Generate Embeddings using sentence-transformer

In [125]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [127]:
device = torch.device("cuda")
model.to(device)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [128]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [129]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [140]:
json_loader = JSONLoader(f"{path}/transcript_json/combined_data.json", jq_schema=".[]", text_content=False)
json_texts = json_loader.load()
embedding = get_embeddings(str(json_texts))
embedding.shape

torch.Size([1, 768])

### Train using HuggingFace Embeddings

In [232]:
hf_embeddings = HuggingFaceEmbeddings()

In [233]:
db = FAISS.from_documents(json_texts, hf_embeddings)

In [234]:
# Embed and chunk transcript
text_splitter = SemanticChunker(hf_embeddings)
split_docs = text_splitter.create_documents([video_transcript])

In [235]:
def merge_short_documents(documents, min_length=2000):
    i = 0
    while i < len(documents) - 1:
        current_doc = documents[i]
        if len(current_doc.page_content) < min_length:
            documents[i + 1].page_content = current_doc.page_content + documents[i + 1].page_content
            del documents[i]
        else:
            i += 1
    return documents

In [236]:
merged_docs = merge_short_documents(split_docs)

### Main Markdown File Prompting with GPT-3.5-Turbo & LangChain Setup

In [237]:
guide_prompt_template = """

Below is a script from a video that I am making into a companion guide blog post first. \
You are a helpful assistant made to assist in the creation I'm doing. \
This is a continuation of a guide so include chapters, key summaries, and incorporate visual aids and direct links to relevant parts of the video, \
however do not include any conclusion or overarching title. \
For visual aids, specific frames from the video will be identified where images can be inserted to enhance understanding. \
For direct links, portions of the text should be hyperlinked to their corresponding times in the video. \
To indicate that a sentence should be hyperlinked, insert the raw text of the transcript next to the word with the indicator <HYPERLINK: "corresponding transcript text">. \
To indicate a picture regarding the text, insert the indicator <PICTURE: "corresponding transcript text">. \
It is crucial to use the raw text from the transcript that will be used, as the additional tools that will be inserting the hyperlinks and pictures need this to know where in the video to look.

In this blog post, in addition to the paragraphs: \

Create titles or headings that encapsulate main points and ideas \

Format your response in markdown, ensuring distinction and clean styling between titles and paragraphs. \
Be sure to include the image placeholders, and hyperlinks with enough distinguishable text WITHOUT ANY QUOTATIONS, as the placeholders will be fed into a semantic search algorithm. \
This structured approach will be applied to the entire transcript. \
The example below only shows one style, but use multiple styles including different headings, bullet points, and other markdown elements when needed. \

Here are shortened example of the input and shortened expected output:

example input:

Hi everyone. So in this video I'd like us to cover the process of tokenization in large language models. Now you see here that I have a sad face and that's because tokenization is my least favorite part of working with large language models but unfortunately it is necessary to understand in some detail because it is fairly hairy, gnarly and there's a lot of hidden foot gums to be aware of and a lot of oddness with large language models typically traces back to tokenization. So what is tokenization? Now in my previous video Let's Build GPT from Scratch we actually already did tokenization but we did a very naive simple version of tokenization. So when you go to the Google Colab for that video you see here that we loaded our training set and our training set was this Shakespeare dataset. Now in the beginning the Shakespeare dataset is just a large string in Python it's just text and so the question is how do we plug text into large language models and in this case here we created a vocabulary of 65 possible characters that we saw occur in this string. These were the possible characters and we saw that there are 65 of them and then we created a lookup table for converting from every possible character a little string piece into a token an integer. So here for example we tokenized the string hi there and we received this sequence of tokens and here we took the first 1000 characters of our dataset and we encoded it into tokens and because this is character level we received 1000 tokens in a sequence so token 18, 47, etc. Now later we saw that the way we plug these tokens into the language model is by using an embedding table and so basically if we have 65 possible tokens then this embedding table is going to have 65 rows and roughly speaking we're taking the integer associated with every single token we're using that as a lookup into this table and we're plucking out the corresponding row and this row is trainable parameters that we're going to train using backpropagation and this is the vector that then feeds into the transformer and that's how the transformer sort of perceives every single token. So here we had a very naive tokenization process that was a character level tokenizer

example output:

Introduction to Tokenization
----------------------------

Welcome to our comprehensive guide on tokenization in large language models (LLMs). Tokenization is a critical yet complex aspect of working with LLMs, essential for understanding how these models process text data. Despite its challenges, tokenization is foundational, as it converts strings of text into sequences of tokens, small units of text that LLMs can manage more effectively.

<PICTURE: Now you see here that I have a sad face and that's because tokenization is my least favorite part of working with large language models but unfortunately it is necessary to understand in some detail because it is fairly hairy, gnarly and there's a lot of hidden foot gums>

Understanding the Basics of Tokenization
----------------------------------------

Tokenization involves creating a vocabulary from all unique characters or words in a dataset and converting each into a corresponding integer token. This process was briefly introduced in our "Let's Build GPT from Scratch" video, where we tokenized a Shakespeare dataset at a character level, creating a vocabulary of 65 possible characters.

<HYPERLINK: So what is tokenization? Now in my previous video Let's Build GPT from Scratch we actually already did tokenization but we did a very naive simple version of tokenization. So when you go to the Google Colab for that video you see here that we loaded>

The Role of Embedding Tables in Tokenization
--------------------------------------------

After tokenization, the next step involves using an embedding table, where each token's integer is used as a lookup to extract a row of trainable parameters. These parameters, once trained, feed into the transformer model, allowing it to perceive each token effectively.

<PICTURE: using backpropagation and this is the vector that then feeds into the transformer and that's how the transformer sort of perceives every single token. So here we had a very naive tokenization process that was a character level tokenizer>

end examples.

Here is the transcript:

{transcript}

"""

output_parser = StrOutputParser()
llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo-0125")
guide_prompt = ChatPromptTemplate.from_template(guide_prompt_template)

guide_chain = (
    {"transcript": RunnablePassthrough()} 
    | guide_prompt
    | llm
    | output_parser
)

def generate_markdown(merged_docs, path, guide_chain):
    markdown_outputs = []
    for doc in merged_docs:
        print(f"*********************************\n{doc}")
        output = guide_chain.invoke(doc.page_content)
        markdown_outputs.append(output)
        time.sleep(1200)
    combined_output = '\n\n'.join(markdown_outputs)
    with open(f'{path}/transcript_json/llm_outline.txt', 'w') as file:
        file.write(combined_output)


In [ ]:
generate_markdown(merged_docs, path, guide_chain)

### Dealing with the Placeholders, Grabbing Pictures & Formatting Hyperlinks

In [239]:
def grab_frame(video, second):
    frames_dir = f'{path}/frames'
    if not os.path.exists(frames_dir):
        os.makedirs(frames_dir)
    cap = cv2.VideoCapture(video)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_number = round(int(second * fps))
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_number >= total_frames:
        print(f"Error: Frame number {frame_number} exceeds total frames in video.")
        cap.release()
        return None

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        cap.release()
        return None

    
    frame_path = os.path.join(frames_dir, f'frame_{second}.jpg')
    cv2.imwrite(frame_path, frame)
    cap.release()
    
    return frame_path

def retrieve_time(segment):
    docs = retriever.get_relevant_documents(segment)
    docs_dict = json.loads(docs[0].page_content)
    start_time = docs_dict["start"]
    end_time = docs_dict["end"]
    time = (start_time + end_time) / 2
    final_time = round(time)
    return final_time

def create_hyperlink(segment, url):
    time = retrieve_time(segment)
    time_link = f"{url}&t={time}s"
    return time_link

def format_seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def process_placeholder(placeholder):
    if placeholder.startswith("<PICTURE:"):
        description = placeholder[9:-1]
        time = retrieve_time(description)
        image_path = grab_frame(video_path, time)
        # Embed the image using markdown with a specified width
        return f'<img src="{image_path}" alt="{description}" width="450"/>'
    elif placeholder.startswith("<HYPERLINK:"):
        text = placeholder[11:-1]
        time = retrieve_time(text)
        formatted_time = format_seconds_to_hms(time)
        hyperlink = create_hyperlink(text, url)
        return f'[Jump to this part of the video: {formatted_time}]({hyperlink})'
    else:
        return placeholder

def replace_placeholders(content):
    placeholders = re.findall(r"<[^>]+>", content)
    for placeholder in placeholders:
        replacement = process_placeholder(placeholder)
        content = content.replace(placeholder, replacement, 1)
    return content

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def convert_txt(path, title, db):
    txt_file_path = f'{path}/transcript_json/llm_outline.txt'
    output_file_path = f'{path}/companion_guide.txt'
    global video_path
    video_path = f'{path}/original_files/video/video_file.mp4'
    global retriever
    retriever = db.as_retriever(search_kwargs={"k": 1})
    
    content = read_file(txt_file_path)
    updated_content = replace_placeholders(content)
    
    with open(output_file_path, 'w') as file:
        file.write(updated_content)
    
    print(f"Updated markdown content has been written to {output_file_path}")


In [240]:
convert_txt(path, title, db)

Updated markdown content has been written to /home/jovyan/work/users/pshalini/video_toBlog_transcriptor/companion_guide.txt


In [194]:
print(db.index.ntotal)

1540


### Generate md file

In [203]:
import shutil
shutil.copyfile("companion_guide.txt", "companion_guide_output.md")

'companion_guide_output.md'